<a href="https://colab.research.google.com/github/gvnbleid/2020-GUM/blob/master/MGU_Projekt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integracja z Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
BASE_DIR = '/content/gdrive/My Drive/DL2020/Projekt2/'

# Sieć konwolucyjna

## Konfiguracja wstępna

In [0]:
%matplotlib inline

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import time
import copy

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cuda not available")
print(device)

### Definicje klas i metod

In [0]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [0]:
def update_classify_table(classify_table, predictions, labels):
        for lab, pred in zip(labels, predictions):
            classify_table[lab, pred] += 1

def train_step(last_epoch):
    running_loss = 0.0
    for i, (img, label) in enumerate(trainloader):
        img = img.to(device)
        label = label.to(device)
        net.train()
        optimizer.zero_grad()
        prediction = net(img)
        loss = None
        if config == "Zmodyfikowany LeNet-5" or config == "Wlasna konfiguracja 1":
            print("dupa")
            loss = criterion(prediction[0], label)
        else:
            print("gowno")
            loss = criterion(prediction, label)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss
    

def print_accuracy():
    correct = 0
    total = 0
    loss = 0.0
    with torch.no_grad():
        net.eval()
        classify_table = np.zeros((10,10))
        for i, (img, label) in enumerate(testloader):
            img = img.to(device)
            label = label.to(device)
            prediction = net(img)
            if config == "Zmodyfikowany LeNet-5" or config == "Wlasna konfiguracja 1":
                loss += criterion(prediction[0], label)
                _ , prediction = torch.max(prediction[0].data, 1)
            else:
                loss += criterion(prediction, label)
                _ , prediction = torch.max(prediction.data, 1)
            update_classify_table(classify_table, prediction, label.data)
            correct += torch.sum(prediction == label.data)

    accuracy = correct.cpu().numpy() / 10000
    print('Accuracy of the network on the 10000 test images: %.2f %%' % (
        100 * accuracy))   
    return accuracy

## Konfiguracja sieci

In [0]:
#@title Wybór konfiguracji { run: "auto" }

config = "Pretrained VGG16" #@param ["Adventures in Machine Learning", "PyTorch tutorial", "Zmodyfikowany LeNet-5", "Wlasna konfiguracja 1", "Pretrained VGG16"]

In [0]:
last_epoch = 0
last_loss = None

class Net(nn.Module):
    if config == "PyTorch tutorial":
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(3, 6, 5)
            self.pool = nn.MaxPool2d(2, 2)
            self.conv2 = nn.Conv2d(6, 16, 5)
            self.fc1 = nn.Linear(16 * 5 * 5, 120)
            self.fc2 = nn.Linear(120, 84)
            self.fc3 = nn.Linear(84, 10)

        def forward(self, x):
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = x.view(-1, 16 * 5 * 5)
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return x

    elif config == "Adventures in Machine Learning":
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(3, 32, 5)
            self.pool = nn.MaxPool2d(2, 2)
            self.conv2 = nn.Conv2d(32, 64, 5)
            self.fc1 = nn.Linear(64 * 5 * 5, 1000)
            self.fc2 = nn.Linear(1000, 10)

        def forward(self, x):
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = x.view(-1, 64 * 5 * 5)
            x = F.relu(self.fc1(x))
            x = F.softmax(self.fc2(x), dim=0)
            return x

    elif config == "Zmodyfikowany LeNet-5":
        def __init__(self):
            super(Net, self).__init__()

            self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 8, stride = 1, kernel_size = 3, padding = 1)
            self.relu1 = nn.ReLU()
            self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, stride = 1, kernel_size = 3, padding = 1)
            self.relu2 = nn.ReLU()
            self.maxpool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
            self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 32, stride = 1, kernel_size = 3, padding = 0)
            self.relu3 = nn.ReLU()
            self.maxpool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
            self.linear1 = nn.Linear(7 * 7 * 32, 300)
            self.relu4 = nn.ReLU()
            self.drop1 = nn.Dropout(p = 0.5)
            self.linear2 = nn.Linear(300, 10)

            self.net = nn.Sequential(self.conv1, self.relu1, self.conv2, self.relu2,
                                 self.maxpool1, self.conv3, self.relu3, self.maxpool2)
                                             
        def forward(self, x):
            x = self.net(x)
            x = x.view(-1, x.shape[0] , 7 * 7 * 32)
            x = self.linear1(x)
            x = self.relu4(x)
            x = self.drop1(x)
            x = self.linear2(x)
            return x

    elif config == "Wlasna konfiguracja 1":
        def __init__(self):
            super(Net, self).__init__()

            self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, stride = 1, kernel_size = 3, padding = 1)
            self.batchnorm1 = nn.BatchNorm2d(num_features = 16)
            self.relu1 = nn.ReLU()
            self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, stride = 1, kernel_size = 3, padding = 1)
            self.batchnorm2 = nn.BatchNorm2d(num_features = 32)
            self.relu2 = nn.ReLU()
            self.maxpool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
            self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, stride = 1, kernel_size = 3, padding = 0)
            self.batchnorm3 = nn.BatchNorm2d(num_features = 64)
            self.relu3 = nn.ReLU()
            self.maxpool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)
            self.linear1 = nn.Linear(7 * 7 * 64, 300)
            self.relu4 = nn.ReLU()
            self.drop1 = nn.Dropout(p = 0.5)
            self.linear2 = nn.Linear(300, 10)
            self.net = nn.Sequential(self.conv1, self.batchnorm1, self.relu1, self.conv2, self.batchnorm2, self.relu2,
                                 self.maxpool2, self.conv3, self.batchnorm3, self.relu3, self.maxpool3)
                                             
        def forward(self, x):
            x = self.net(x)
            x = x.view(-1, x.shape[0] , 7 * 7 * 64)
            x = self.linear1(x)
            x = self.relu4(x)
            x = self.drop1(x)
            x = self.linear2(x)
            return x

net = None
input_size = 32

if config == "Pretrained VGG16":
    def set_parameter_requires_grad(model):
        for param in model.parameters():
            param.requires_grad = False

    net = models.vgg16_bn(pretrained=True)
    set_parameter_requires_grad(net)
    num_ftrs = net.classifier[6].in_features
    net.classifier[6] = nn.Linear(num_ftrs, 10)
    net = net.to(device)
    input_size = 224
else:
    net = Net().to(device)

net

In [0]:
from torchsummary import summary
summary(net, input_size=(3, input_size, input_size))

### Wczytanie stanu sieci (opcjonalne)

In [0]:
load_state = True #@param {type: "boolean"}
path = "wlasna_best_20.04.19_15:40:35.pt" #@param {type: "string"}

if load_state:
    checkpoint = torch.load(BASE_DIR + path)
    net.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    criterion.load_state_dict(checkpoint['criterion_state_dict'])
    last_epoch = checkpoint['epoch']
    last_loss = checkpoint['loss']
    loss_array = checkpoint['loss_array']
    acc_array = checkpoint['acc_array']
    net.eval()

## Ustawienia augmentacji danych

In [0]:
#@title Metody augmentacji { run: "auto" }
translation_checkbox = False #@param {type:"boolean"}
flip_checkbox = True #@param {type:"boolean"}
rotation_checkbox = False #@param {type:"boolean"}
noise_checkbox = False #@param {type:"boolean"}
color_checkbox = True #@param {type:"boolean"}
crop_checkbox = True #@param {type:"boolean"}

In [0]:
transforms_array_train = []

transforms_array_train.append(transforms.Resize(size=(input_size, input_size)))

if crop_checkbox:
    transforms_array_train.append(transforms.RandomCrop(padding=None, size=(input_size, input_size)))
if translation_checkbox:
    transforms_array_train.append(transforms.RandomAffine(0, (0.2, 0.2)))
if flip_checkbox:
    transforms_array_train.append(transforms.RandomHorizontalFlip(p=0.5))
if rotation_checkbox:
    transforms_array_train.append(transforms.RandomRotation(degrees=(-15,15), resample=False, expand=False))
if color_checkbox:
    transforms_array_train.append(transforms.ColorJitter(brightness=[0.8,1.2], contrast=[0.8,1.2], saturation=[0.8,1.2]))
if noise_checkbox:
    transforms_array_train.append(AddGaussianNoise(0., 0.2))

transforms_array_train.append(transforms.ToTensor())
transforms_array_train.append(transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                            std=[0.229, 0.224, 0.225]))
transforms_array_test = []

transforms_array_test.append(transforms.Resize(size=(input_size, input_size)))
transforms_array_test.append(transforms.ToTensor())
transforms_array_test.append(transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225]))

## Wczytanie danych

In [0]:
transform_train = transforms.Compose(transforms_array_train)
transform_test = transforms.Compose(transforms_array_test)

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(10)))

## Uczenie sieci - funkcja

In [0]:
nb_epoch = 10 #@param {type: "integer"}

In [0]:
def train_net(net, dataloaders, criterion, optimizer, nb_epoch, last_epoch):
    since = time.time()

    test_acc_array = []
    train_acc_array = []
    loss_array = []
    best_net = copy.deepcopy(net.state_dict())
    best_acc = 0.0

    for epoch in range(nb_epoch):  # loop over the dataset multiple times
        last_epoch = last_epoch + 1

        for phase in ['train', 'test']:
            if phase == 'train':
                net.train()
                dataloader = dataloaders[0]
            else:
                net.eval()
                dataloader = dataloaders[1]
        
            running_loss = 0.0
            running_correct = 0

            for inputs, labels in dataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    # forward + backward + optimize
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)

                    _, prediction = torch.max(outputs, 1)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # print statistics
                running_loss += loss.item() * inputs.size(0)
                running_correct += torch.sum(prediction == labels.data)

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_correct.double() / len(dataloader.dataset)

            print('Epoch %s/%s - %s Loss: %.3f Acc: %.3f' %
                (epoch + 1, nb_epoch, phase, epoch_loss, epoch_acc))
            
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_net = copy.deepcopy(net.state_dict())
            if phase == 'test':
                test_acc_array.append(epoch_acc)
            else:
                train_acc_array.append(epoch_acc)
            loss_array.append(epoch_loss)
    
    training_time = time.time() - since
    print('Finished Training in %.0fm %.0fs' % (training_time // 60, training_time % 60))
    print('Best test Accuracy: %.3f' % (best_acc))

    net.load_state_dict(best_net)
    return net, test_acc_array, train_acc_array, loss_array

net, test_acc_array, train_acc_array, loss_array = train_net(net, [trainloader, testloader], criterion, optimizer, nb_epoch, last_epoch)

## Zapis stanu modelu

In [0]:
from datetime import datetime

def append_timestamp(path):
    now = datetime.now()
    current_time = now.strftime("%y.%m.%d_%H:%M:%S")
    return path.replace(".", f"_{current_time}.")

In [0]:
def save_model():
    save_state = True #@param {type: "boolean"}
    path = "wlasna_best.pt" #@param {type: "string"}
    use_timestamp = True #@param {type: "boolean"}

    if save_state:
        if use_timestamp:
            path = append_timestamp(path)
        torch.save({
                    'epoch': last_epoch,
                    'model_state_dict': net.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'criterion_state_dict': criterion.state_dict(),
                    'loss': last_loss,
                    'loss_array': loss_array,
                    'acc_array': acc_array
                    }, BASE_DIR + path)

## Uczenie sieci

In [0]:
import torch.optim as optim

optimizer_type = "SGD" #@param ["SGD", "Adam"]
lr = 0.001 #@param {type: "number"}

criterion = nn.CrossEntropyLoss()

params_to_update = []
for param in net.parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

optimizer = None

if optimizer_type == "SGD":
    optimizer = optim.SGD(params_to_update, lr=lr, momentum=0.9)
if optimizer_type == "Adam":
    optimizer = optim.Adam(params_to_update, lr = 0.000002, weight_decay=0.002)

def update_classify_table(classify_table, predictions, labels):
        for lab, pred in zip(labels, predictions):
            classify_table[lab, pred] += 1

def train_step(last_epoch):
    running_loss = 0.0
    for i, (img, label) in enumerate(trainloader):
        img = img.to(device)
        label = label.to(device)
        net.train()
        optimizer.zero_grad()
        prediction = net(img)
        loss = None
        if config == "Zmodyfikowany LeNet-5" or config == "Wlasna konfiguracja 1":
            loss = criterion(prediction[0], label)
        else:
            loss = criterion(prediction, label)
        loss.backward()
        optimizer.step()

        if config == "Zmodyfikowany LeNet-5" or config == "Wlasna konfiguracja 1":
            running_loss += loss.item()
        else:
            running_loss += loss.item() * img.size(0)
    return running_loss
    

def print_accuracy():
    correct = 0
    total = 0
    loss = 0.0
    with torch.no_grad():
        net.eval()
        classify_table = np.zeros((10,10))
        for i, (img, label) in enumerate(testloader):
            img = img.to(device)
            label = label.to(device)
            prediction = net(img)
            if config == "Zmodyfikowany LeNet-5" or config == "Wlasna konfiguracja 1":
                loss += criterion(prediction[0], label)
                _ , prediction = torch.max(prediction[0].data, 1)
            else:
                loss += criterion(prediction, label)
                _ , prediction = torch.max(prediction.data, 1)
            update_classify_table(classify_table, prediction, label.data)
            correct += torch.sum(prediction == label.data)

    accuracy = correct.cpu().numpy() / 10000
    print('Accuracy of the network on the 10000 test images: %.2f %%' % (
        100 * accuracy))   
    return accuracy

In [0]:
nb_epoch =  3#@param {type: "integer"}

In [0]:
counter = 0
best_net = copy.deepcopy(net.state_dict())

loss_array = []
acc_array = []
max_acc = 0

for e in range(nb_epoch):
    last_epoch = last_epoch + 1
    running_loss = train_step(last_epoch)
    if config == "Zmodyfikowany LeNet-5" or config == "Wlasna konfiguracja 1":
        last_loss = running_loss / 500
    else:
        last_loss = running_loss / len(trainloader.dataset)
    print('%d epoch, loss: %.4f' % (last_epoch, last_loss))
    loss_array.append(last_loss)
    curr_acc = print_accuracy()
    acc_array.append(curr_acc)
    if curr_acc > max_acc:
        #save_model()
        best_net = copy.deepcopy(net.state_dict())
        max_acc = curr_acc
        counter = 0
    else:
        counter = counter + 1
        if counter > 2:
            break

net.load_state_dict(best_net)

## Wyniki

In [0]:
correct = 0
total = 0
loss = 0.0
with torch.no_grad():
    net.eval()
    classify_table = np.zeros((10,10))
    for i, (img, label) in enumerate(testloader):
        img = img.to(device)
        label = label.to(device)
        prediction = net(img)
        if config == "Zmodyfikowany LeNet-5" or config == "Wlasna konfiguracja 1":
            loss += criterion(prediction[0], label)
            _ , prediction = torch.max(prediction[0].data, 1)
        else:
            loss += criterion(prediction, label)
            _ , prediction = torch.max(prediction.data, 1)

        update_classify_table(classify_table, prediction, label.data)
        correct += torch.sum(prediction == label.data)

accuracy = correct.cpu().numpy() / 10000
print('Accuracy of the network on the 10000 test images: %.2f %%' % (
    100 * accuracy)) 

### Zapis wyników (opcjonalne)

In [0]:
save_results_state = True #@param {type: "boolean"}
use_timestamp = True #@param {type: "boolean"}
path = "pytorch_tutorial.json" #@param {type: "string"}

from datetime import datetime
import json
import codecs

if save_results_state:
    if use_timestamp:
        path = append_timestamp(path)

    json.dump(dict(accuracy=accuracy, classify_table=accuracy_per_class),
              codecs.open(BASE_DIR + path, 'w', encoding='utf-8'))

## Wykresy

In [0]:
import matplotlib.pyplot as plt

epochs = range(1, last_epoch + 1)

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.set_xticks(np.arange(0, last_epoch + 1, 5))

plt.scatter(epochs, loss_array)
plt.title("Wykres funkcji straty dla zbioru treningowego")
plt.xlabel("Numer epoki")
plt.ylabel("Wartość funkcji straty")
plt.show()

In [0]:
import matplotlib.pyplot as plt

epochs = range(1, last_epoch + 1)

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.set_xticks(np.arange(0, last_epoch + 1))

plt.scatter(epochs, acc_array)
plt.title("Wykres dokładności dla zbioru walidacyjnego")
plt.xlabel("Numer epoki")
plt.ylabel("Dokładność")
plt.show()

In [0]:
import copy
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    
def visualise_accuracy_by_class(classify_table):
    results = [ classify_table[i,i] / np.sum(classify_table[i, :]) for i in range(10)]

    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])

    plt.bar(classes, results, color = ['#7e57c2', '#ffc400'])
    plt.title("Frakcja poprawnych klasyfikacji dla poszczególnych klas")
    plt.xlabel('Klasa')
    plt.ylabel('Frakcja poprawnych klasyfikacji')
    plt.xticks(classes)
    plt.show()
    
def visualise_errors_by_class(classify_table):
    p = list()
    table = copy.deepcopy(classify_table)
    table[np.argmax(table, 0), np.argmax(table, 1)] = 0
    p.append(plt.bar(classes, table[:, 0]))
    for i in range(1, 10):
        p.append(plt.bar(classes, table[:, i], bottom = np.sum(table[:, 0:i], 1)))

    plt.title("Błędy klasyfikacji")
    plt.xticks(classes)
    plt.xlabel("Poprawna klasa")
    plt.ylabel("Liczba błędnych klasyfikacji")
    plt.legend(classes, title = "Klasa zwracana przez sieć", bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()

def visualise_errors_for_class(classify_table, class_index):
    p = list()
    table = copy.deepcopy(classify_table)
    table[np.argmax(table, 0), np.argmax(table, 1)] = 0

    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])

    plt.bar(classes, table[:, class_index])
    plt.xticks(classes)
    plt.title("Liczba błędnych klasyfikacji dla klasy: {}".format(classes[class_index]))
    plt.xlabel("Klasa zwracana przez sieć")
    plt.ylabel("Liczba błędnych klasyfikacji")
    plt.show()

In [0]:
visualise_accuracy_by_class(classify_table)

In [0]:
visualise_errors_by_class(classify_table)

In [0]:
visualise_errors_for_class(classify_table, 3)